# Changes made by kts4 / noami2
- `glove` was not available and so needed to install `mittens` instead
  - `from mittens import GloVe as glove`
- Removed `merge` from import from `keras.layers`
  - No longer available.
  - Used `Concatenate()([x, text_embeddings])` instead of `merge([x, text_embeddings], mode='concat', concat_axis=1)`
  - Code was available already, just commented out.
- `set_session`, `clear_session` and `get_session` are no longer available from `keras.backend.tensorflow_backend`
  - Loaded from `tf.compat.v1.keras.backend` and `tf.keras.backend` instead
- Imported `Path`
  - Allows us to ensure the required directories for saving are available
- Allow user to select which embedding techniques to calculate
- Allow user to select whether to run ablation or not
  - Update CNN if ablation is set to `True`
  - Added input to `save_scores_cnn` to allow to save ablation results separately
- Updated `get_subvector_data` and `proposedmodel` to set appropriate dimensions for ablations
- `tf.contrib.layers.l2_regularizer` no longer available
  - Used `tf.keras.regularizers.l2` instead
- `tf.contrib.layers.xavier_initializer` no longer available
  - Used `tf.keras.initializers.GlorotUniform` instead
- Changed `x_train_ner = np.asarray(x_train_dict_sorted.values())` to `x_train_ner = np.array(list(x_train_dict_sorted.values()))`
  - Change due to Python 3
- Changes to newlines / spacings / printing etc.

In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
from mittens import GloVe as glove

import collections
import gc 

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate, GRU, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, Convolution1D, UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, MaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import clear_session

from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

from pathlib import Path
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
run_word2vec     = True
run_fastText     = True
run_combined     = True
run_blueBERT     = False
run_clinicalBERT = False

run_ablation     = False

In [3]:
x_train = pd.read_pickle("data/new_x_train.pkl")
x_dev   = pd.read_pickle("data/new_x_dev.pkl")
x_test  = pd.read_pickle("data/new_x_test.pkl")

y_train = pd.read_pickle("data/new_y_train.pkl")
y_dev   = pd.read_pickle("data/new_y_dev.pkl")
y_test  = pd.read_pickle("data/new_y_test.pkl")


if run_word2vec:     ner_word2vec     = pd.read_pickle("data/new_ner_word2vec_limited_dict.pkl")
if run_fastText:     ner_fasttext     = pd.read_pickle("data/new_ner_fasttext_limited_dict.pkl")
if run_combined:     ner_concat       = pd.read_pickle("data/new_ner_combined_limited_dict.pkl")
if run_clinicalBERT: ner_clinicalBERT = pd.read_pickle("data/new_ner_clinicalbert_limited_dict.pkl")
if run_blueBERT:     ner_blueBERT     = pd.read_pickle("data/new_ner_bluebert_limited_dict.pkl")

train_ids = pd.read_pickle("data/new_train_ids.pkl")
dev_ids   = pd.read_pickle("data/new_dev_ids.pkl")
test_ids  = pd.read_pickle("data/new_test_ids.pkl")

In [4]:
def make_prediction_cnn(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_cnn(predictions, 
                    probs, 
                    ground_truth, 
                    embed_name, 
                    problem_type, 
                    iteration, 
                    hidden_unit_size,
                    sequence_name,
                    ablation_name=""):
    
    auc   = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict          = {}    
    result_dict['auc']   = auc
    result_dict['auprc'] = auprc
    result_dict['acc']   = acc
    result_dict['F1']    = F1

    result_path = "results/09-cnn/"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name+ablation_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-new-cnn-.p"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))
    
def print_scores_cnn(predictions, probs, ground_truth, model_name, problem_type, iteration, hidden_unit_size):
    auc   = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    print ("AUC: ", auc, "AUPRC: ", auprc, "F1: ", F1)
    
def get_subvector_data(size, embed_name, data):
    if embed_name == "concat":
        vector_size = 200
    elif embed_name == "clinicalBERT":
        vector_size = 768
    elif embed_name == "blueBERT":
        vector_size = 768
    else:
        vector_size = 100

    x_data = {}
    for k, v in data.items():
        number_of_additional_vector = len(v) - size
        vector = []
        for i in v:
            vector.append(i)
            
        if number_of_additional_vector < 0: 
            number_of_additional_vector = np.abs(number_of_additional_vector)

            temp = vector[:size]
            for i in range(0, number_of_additional_vector):
                temp.append(np.zeros(vector_size))
            x_data[k] = np.asarray(temp)
        else:
            x_data[k] = np.asarray(vector[:size])

    return x_data


def proposedmodel(layer_name, number_of_unit, embedding_name, ner_limit, num_filter, run_ablation):
    if embedding_name == "concat":
        input_dimension = 200
    elif embedding_name == "clinicalBERT":
        input_dimension = 768
    elif embedding_name == "blueBERT":
        input_dimension = 768
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_img = Input(shape=(ner_limit, input_dimension), 
                      name = "cnn_input")

    convs = []
    filter_sizes = [2,3,4]

    text_conv1d = Conv1D(filters=num_filter, 
                         kernel_size=3, 
                         padding = 'valid', 
                         strides = 1, 
                         dilation_rate=1, 
                         activation='relu', 
                         kernel_initializer=tf.keras.initializers.GlorotUniform()
                        )(input_img)
    if run_ablation:
        text_conv1d = Dropout(0.2)(text_conv1d)
    
    text_conv1d = Conv1D(filters=num_filter*2, 
                         kernel_size=3, 
                         padding = 'valid', 
                         strides = 1, 
                         dilation_rate=1, 
                         activation='relu',
                         kernel_initializer=tf.keras.initializers.GlorotUniform()
                        )(text_conv1d)   
    if run_ablation:
        text_conv1d = Dropout(0.2)(text_conv1d)
    
    text_conv1d = Conv1D(filters=num_filter*3, 
                         kernel_size=3, 
                         padding = 'valid', 
                         strides = 1, 
                         dilation_rate=1, 
                         activation='relu',
                         kernel_initializer=tf.keras.initializers.GlorotUniform()
                        )(text_conv1d)   
    
    if run_ablation:
        text_conv1d = Dropout(0.2)(text_conv1d)
    
        text_conv1d = Conv1D(filters=num_filter*4, 
                             kernel_size=3, 
                             padding = 'valid', 
                             strides = 1, 
                             dilation_rate=1, 
                             activation='relu',
                             kernel_initializer=tf.keras.initializers.GlorotUniform()
                            )(text_conv1d)   
        text_conv1d = Dropout(0.2)(text_conv1d)

        text_conv1d = Conv1D(filters=num_filter*5, 
                             kernel_size=3, 
                             padding = 'valid', 
                             strides = 1, 
                             dilation_rate=1, 
                             activation='relu',
                             kernel_initializer=tf.keras.initializers.GlorotUniform()
                            )(text_conv1d)   
        text_conv1d = Dropout(0.2)(text_conv1d) 

    
    text_embeddings = GlobalMaxPooling1D()(text_conv1d)
    
    x = GRU(number_of_unit)(sequence_input)
    
    concatenated = Concatenate()([x, text_embeddings])

    concatenated = Dense(512, activation='relu')(concatenated)
    concatenated = Dropout(0.2)(concatenated)
    
    logits_regularizer = tf.keras.regularizers.L2(l2=0.01)
    preds = Dense(1, 
                  activation='sigmoid',
                  use_bias=False,
                  kernel_initializer=tf.keras.initializers.GlorotUniform(), 
                  kernel_regularizer=logits_regularizer
                 )(concatenated)
    
    opt = Adam(learning_rate=1e-3, decay = 0.01)

    model = Model(inputs=[sequence_input, input_img], 
                  outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
    
    return model


In [5]:
embedding_types = []
embedding_dict  = []

if run_word2vec:
    embedding_types.append('word2vec')
    embedding_dict.append(ner_word2vec)

if run_fastText:
    embedding_types.append('fasttext')
    embedding_dict.append(ner_fasttext)

if run_combined:
    embedding_types.append('concat')
    embedding_dict.append(ner_concat)

if run_clinicalBERT:
    embedding_types.append('clinicalBERT')
    embedding_dict.append(ner_clinicalBERT)

if run_blueBERT:
    embedding_types.append('blueBERT')
    embedding_dict.append(ner_blueBERT)

target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
ablation_name   = "_cnn_L5"

num_epoch        = 100
model_patience   = 5
monitor_criteria = 'val_loss'
batch_size       = 64

filter_number    = 32
ner_representation_limit = 64
activation_func  = "relu"

sequence_model   = "GRU"
sequence_hidden_unit = 256

maxiter = 11
for embed_dict, embed_name in zip(embedding_dict, embedding_types):  
    
    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    temp_dev_ner   = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner  = dict((k, embed_dict[k]) for k in test_ids)

    x_train_dict = {}
    x_dev_dict   = {}
    x_test_dict  = {}

    x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
    x_dev_dict   = get_subvector_data(ner_representation_limit, embed_name, temp_dev_ner)
    x_test_dict  = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

    x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
    
    x_dev_dict_sorted   = collections.OrderedDict(sorted(x_dev_dict.items()))
    x_test_dict_sorted  = collections.OrderedDict(sorted(x_test_dict.items()))

    x_train_ner = np.asarray(list(x_train_dict_sorted.values()))
    x_dev_ner   = np.asarray(list(x_dev_dict_sorted.values()))
    x_test_ner  = np.asarray(list(x_test_dict_sorted.values()))
        
    for iteration in tqdm(range(1,maxiter)):
        for each_problem in target_problems:  
            
            print ("Embedding: ", embed_name)
            print ("Iteration number: ", iteration)
            print ("Problem type: ", each_problem)
            print ("__________________")
            
            
            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, 
                                                   patience=model_patience)
            
            best_model_name = "results/Best Models/" + str(ner_representation_limit)+"-basiccnn1d-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.keras"
            
            checkpoint = ModelCheckpoint(best_model_name, 
                                         monitor=monitor_criteria, 
                                         verbose=0,
                                         save_best_only=True, 
                                         mode='min')
            
            reduce_lr = ReduceLROnPlateau(monitor=monitor_criteria, 
                                          factor=0.2,
                                          patience=2, 
                                          min_lr=0.00001, 
                                          epsilon=1e-4, 
                                          mode='min')
            

            callbacks = [early_stopping_monitor, checkpoint, reduce_lr]
            
            model = proposedmodel(sequence_model, 
                                  sequence_hidden_unit, 
                                  embed_name, 
                                  ner_representation_limit,
                                  filter_number,
                                  run_ablation)
            
            model.fit([x_train, x_train_ner], 
                      y_train[each_problem], 
                      epochs=num_epoch, 
                      verbose=0, 
                      validation_data=([x_dev, x_dev_ner], y_dev[each_problem]), 
                      callbacks=callbacks, 
                      batch_size=batch_size)
            
            
            probs, predictions = make_prediction_cnn(model, [x_test, x_test_ner])
            print_scores_cnn(predictions, 
                             probs, 
                             y_test[each_problem], 
                             embed_name, 
                             each_problem, 
                             iteration, 
                             sequence_hidden_unit)
            
            model.load_weights(best_model_name)
                      
            probs, predictions = make_prediction_cnn(model, [x_test, x_test_ner])
            save_scores_cnn(predictions, 
                            probs, 
                            y_test[each_problem], 
                            embed_name, 
                            each_problem, 
                            iteration,
                            sequence_hidden_unit, 
                            sequence_model,
                            ablation_name
                            )
            del model
            clear_session()
            gc.collect()
            
    

  0%|                                                    | 0/10 [00:00<?, ?it/s]2024-04-08 19:52:10.684784: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-04-08 19:52:10.684808: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-04-08 19:52:10.684814: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-04-08 19:52:10.684829: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-08 19:52:10.684848: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________


2024-04-08 19:52:11.623388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
AUC:  0.8642768070942775 AUPRC:  0.5671772439417813 F1:  0.4945945945945946
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  word2vec
Iteration number:  1
Problem type:  mort_icu
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
AUC:  0.8765847442938275 AUPRC:  0.5185406294213413 F1:  0.4775510204081632
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  word2vec
Iteration number:  1
Problem type:  los_3
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
AUC:  0.6731656916858805 AUPRC:  0.6092714276602098 F1:  0.5607843137254903
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  word2vec
Iteration number:  1
Problem type:  los_7
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
AUC:  0.686982010146831 AUPRC:  0.19094184500160882 F1:  0.12669683257918551
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


 10%|████▎                                      | 1/10 [01:50<16:35, 110.66s/it]

...

 80%|██████████████████████████████████▍        | 8/10 [18:08<04:33, 136.93s/it]

Embedding:  concat
Iteration number:  9
Problem type:  mort_hosp
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
AUC:  0.8581051616901151 AUPRC:  0.5484869007526046 F1:  0.4728682170542635
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  concat
Iteration number:  9
Problem type:  mort_icu
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
AUC:  0.8654651836421754 AUPRC:  0.4947141229910811 F1:  0.4618473895582329
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  concat
Iteration number:  9
Problem type:  los_3
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
AUC:  0.6933849512086162 AUPRC:  0.629553670909073 F1:  0.5689753863766456
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  concat
Iteration number:  9
Problem type:  los_7
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
AUC:  0.6997312922172658 AUPRC:  0.18714504230975273 F1:  0.15546218487394955
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


 90%|██████████████████████████████████████▋    | 9/10 [20:25<02:17, 137.08s/it]

Embedding:  concat
Iteration number:  10
Problem type:  mort_hosp
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
AUC:  0.8634893676549825 AUPRC:  0.5639740760080336 F1:  0.46858638743455494
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  concat
Iteration number:  10
Problem type:  mort_icu
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
AUC:  0.8604121154960237 AUPRC:  0.4924026185938085 F1:  0.4639376218323587
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  concat
Iteration number:  10
Problem type:  los_3
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
AUC:  0.6851429230792032 AUPRC:  0.6215515521416118 F1:  0.5842205842205842
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Embedding:  concat
Iteration number:  10
Problem type:  los_7
__________________
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
AUC:  0.7188597613942032 AUPRC:  0.21561130917831334 F1:  0.10864197530864197
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


100%|██████████████████████████████████████████| 10/10 [22:36<00:00, 135.62s/it]
